In [ ]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

mapping = {
    "settings": {
        "analysis": {
            "analyzer": {
                "no_accent_analyzer": {
                    "tokenizer": "standard",
                    "filter": ["lowercase", "asciifolding"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "id": {"type": "keyword"},
            "productname": {
                "type": "text",
                "fields": {
                    "keyword": {"type": "keyword"},
                    "no_accent": {
                        "type": "text",
                        "analyzer": "no_accent_analyzer"
                    }
                }
            },
            "embedding": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            }
        }
    }
}

es.indices.close(index="index-demo2")
es.indices.put_settings(index="index-demo2", body=mapping["settings"])
es.indices.put_mapping(index="index-demo2", body=mapping["mappings"])
es.indices.open(index="index-demo2")

In [1]:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
import numpy as np

es = Elasticsearch("http://localhost:9200")
model = SentenceTransformer('keepitreal/vietnamese-sbert')

C:\Users\ABC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('keepitreal/vietnamese-sbert')

docs = es.search(index="index-demo2", body={"query": {"match_all": {}}}, size=10000)["hits"]["hits"]

for doc in docs:
    doc_id = doc["_id"]
    product_name = doc["_source"]["productname"]
    
    embedding = model.encode(product_name, normalize_embeddings=True)
    
    es.update(
        index="index-demo2",
        id=doc_id,
        body={"doc": {"embedding": embedding.tolist()}}
    )

In [126]:
def full_text_search(query_text, es_client, index_name="index-demo2", top_k=10):

    search_body = {
        "size": top_k,
        "query": {
            "multi_match": {
                "query": query_text,
                "fields": ["productname"],
                "type": "best_fields"
            }
        }
    }
    
    return es_client.search(index=index_name, body=search_body)

query_text = "cá bống"
response = full_text_search(query_text, es)

if response["hits"]["hits"]:
    print(f"Tìm thấy {len(response['hits']['hits'])} kết quả cho '{query_text}':(full-text search)\n")
    for i, hit in enumerate(response["hits"]["hits"], 1):
        print(f"#{i} | Score: {hit['_score']:.4f}")
        print(f"   Tên sản phẩm: {hit['_source'].get('productname', 'N/A')}")
else:
    print("Không tìm thấy kết quả phù hợp")

Tìm thấy 10 kết quả cho 'cá bống':(full-text search)

#1 | Score: 12.7337
   Tên sản phẩm: Cá bống mối XICHMAFOOD khay 500gr +/- 20gr
#2 | Score: 11.2838
   Tên sản phẩm: Cá bống tẩm ghép oval nướng cán HaiNamFoods khay 70g
#3 | Score: 11.2838
   Tên sản phẩm: Cá Bống Đục Làm Sạch Tuấn Nguyễn khay 300g (+/-15g)
#4 | Score: 10.1304
   Tên sản phẩm: Cá bống mối VAN khay 300gr ( mạ băng 25% giữ độ tươi ngon )
#5 | Score: 9.7965
   Tên sản phẩm:  Bánh Orion Con Cá Bống Bang Mochi Socola Đậu Đỏ gói 145g (5P*29g)
#6 | Score: 9.7965
   Tên sản phẩm: [ GIÁ RẺ VÔ ĐỊCH ] Cá bống mối biển nguyên con TPFood khay 300gr +/-20gr
#7 | Score: 9.7965
   Tên sản phẩm: Bánh con cá Bống Bang Mochi Socola đậu đỏ Orion gói 145gr (5gói*29gr)
#8 | Score: 9.7965
   Tên sản phẩm: [TẾT] Bánh Orion Con Cá Bống Bang Mochi Socola Đậu Đỏ gói 348g 12P
#9 | Score: 9.4840
   Tên sản phẩm: [COMBO 3 GÓI] Bánh Orion Con Cá Bống Bang Mochi Socola Đậu Đỏ gói 145g
#10 | Score: 9.4840
   Tên sản phẩm: QT Bánh con cá Bống Bang 

In [ ]:
def knn_search(query_text, es_client, index_name="index-demo2", top_k=10):

    query_embedding = model.encode(query_text).tolist()

    search_body = {
        "query": {
            "knn": {
                "field": "embedding",  
                "query_vector": query_embedding,  
                "num_candidates": top_k  
            }
        },
        "_source": ["id", "productname"],  
    }

    return es_client.search(index=index_name, body=search_body)

query_text = "rau muống"
response = knn_search(query_text, es)

if response["hits"]["hits"]:
    print(f"Tìm thấy {len(response['hits']['hits'])} kết quả cho '{query_text}':'(Semantic search)'\n")
    for i, hit in enumerate(response["hits"]["hits"], 1):
        print(f"#{i} | Score: {hit['_score']:.4f}")
        print(f"   Tên sản phẩm: {hit['_source']['productname']}")
else:
    print("Không tìm thấy kết quả phù hợp")


Tìm thấy 10 kết quả cho 'rau muống':'(Semantic search)'

#1 | Score: 0.8824
   Tên sản phẩm: Rau muống túi 400gr
#2 | Score: 0.8193
   Tên sản phẩm: Rau muống giòn túi 400gr SAPA Lâm Anh
#3 | Score: 0.8189
   Tên sản phẩm: Rau dền túi 300gr
#4 | Score: 0.7976
   Tên sản phẩm: Rau cải bó xôi thủy canh 300gr
#5 | Score: 0.7970
   Tên sản phẩm: [XẢ TỒN] Rau muống hạt túi 400g
#6 | Score: 0.7965
   Tên sản phẩm: Cải canh (cải xanh) túi 400gr
#7 | Score: 0.7963
   Tên sản phẩm: [XẢ TỒN] Rau muống hạt túi 300g
#8 | Score: 0.7928
   Tên sản phẩm: Rau Muống cọng (không lá) túi 300g
#9 | Score: 0.7827
   Tên sản phẩm: Cải xoong túi 400gr
#10 | Score: 0.7805
   Tên sản phẩm: Cải bó xôi túi 400gr


In [132]:
def hybrid_search(query_text, es_client, index_name="index-demo2", top_k=10):

    query_embedding = model.encode(query_text).tolist()

    search_body = {
        "size": top_k * 2, 
        "query": {
            "bool": {
                "should": [
                    {
                        "multi_match": {
                            "query": query_text,
                            "fields": ["productname","productname.no_accent"],
                            "type": "best_fields",
                            "boost": 0.1
                        }
                    },
                    {
                        "knn": {
                            "field": "embedding",
                            "query_vector": query_embedding,
                            "num_candidates": 100,
                            "boost": 0.9
                        }
                    }
                ],
                "minimum_should_match": 1
            }
        },
        "rescore": {
            "window_size": top_k * 2, 
            "query": {
                "rescore_query": {
                    "script_score": {
                        "query": {"match_all": {}},
                        "script": {
                            "source": """
                                double text_score = _score;
                                double semantic_score = 0.0;
                                if (doc['embedding'].size() == params.query_vector.size()) {
                                    semantic_score = cosineSimilarity(params.query_vector, 'embedding') + 1.0;
                                }
                                return text_score * 0.3 + semantic_score * 0.7;
                            """,
                            "params": {"query_vector": query_embedding}
                        }
                    }
                },
                "query_weight": 0.4, 
                "rescore_query_weight": 0.6
            }
        },
        "_source": ["id", "productname","productname.no_accent"]
    }

    return es_client.search(index=index_name, body=search_body)

query_text = "cá diêu hồng"
response = hybrid_search(query_text, es)

if response["hits"]["hits"]:
    print(f"Tìm thấy {len(response['hits']['hits'])} kết quả cho '{query_text}':\n")
    for i, hit in enumerate(response["hits"]["hits"], 1):
        print(f"#{i} | Score: {hit['_score']:.4f}")
        print(f"   Tên sản phẩm: {hit['_source'].get('productname', 'N/A')}")
else:
    print("Không tìm thấy kết quả phù hợp")

Tìm thấy 20 kết quả cho 'cá diêu hồng':

#1 | Score: 1.0751
   Tên sản phẩm: Cá diêu hồng làm sạch Alo Fish khay 300-400g
#2 | Score: 1.0585
   Tên sản phẩm: Cá Diêu Hồng Làm sạch Tuấn Nguyễn túi 500-600g
#3 | Score: 1.0398
   Tên sản phẩm: Cá diêu hồng tươi phi lê Tuấn Nguyễn khay 300gr +/- 15gr
#4 | Score: 0.9943
   Tên sản phẩm: Cá diêu hồng nguyên con làm sạch Tuấn Nguyễn khay 1 con 500-700gr
#5 | Score: 0.9897
   Tên sản phẩm: [DÙNG NGAY - Date 17/03/2025] Cá diêu hồng làm sạch Alo Fish
#6 | Score: 0.8260
   Tên sản phẩm: Cá Đổng/cá Hồng làm sạch HTKFOOD khay 500gr +/-25gr
#7 | Score: 0.7752
   Tên sản phẩm: Cá hồng phèn tươi Thái Bình khay 500gr +/- 25gr
#8 | Score: 0.7575
   Tên sản phẩm: [RẺ] Cá diêu hồng làm sạch Alo Fish khay 300-400g
#9 | Score: 0.7451
   Tên sản phẩm: Cá chim hồng Tuấn Nguyễn làm sạch gói 1 con từ 500-600gr
#10 | Score: 0.7372
   Tên sản phẩm: [CỰC RẺ] Cá diêu hồng làm sạch Alo Fish khay 300-400g
#11 | Score: 0.7372
   Tên sản phẩm: [SIÊU SALE] Cá diêu hồng

In [110]:
import ollama

def rewrite_query_with_llm(query_text, model="llama3"):
    prompt = f"""You are an e-commerce product search query optimization assistant. 
Analyze the query: "{query_text}" 
and add 1-2 words BEFORE or AFTER for better context (e.g. brand, function, features, etc.). 
IMPORTANT: 
1. Only return 1 improved versions of the query on 1 line 
2. Keep the Vietnamese language 
3. No further explanations or comments.
4. Can be added or not added if the query is semantic enough."""

    response = ollama.chat(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    
    rewritten_query = response['message']['content'].strip()
    return rewritten_query


In [111]:
def hybrid_search_with_llm(query_text, es_client, index_name="index-demo2", top_k=10):
    rewritten_query = rewrite_query_with_llm(query_text)
    print(f">>> Query sau khi LLM rewrite: {rewritten_query}")

    query_embedding = model.encode(rewritten_query).tolist()

    search_body = {
        "size": top_k * 2,
        "query": {
            "bool": {
                "should": [
                    {
                        "multi_match": {
                            "query": rewritten_query,
                            "fields": ["productname","productname.no_accent"],
                            "type": "best_fields",
                            "boost": 0.15
                        }
                    },
                    {
                        "knn": {
                            "field": "embedding",
                            "query_vector": query_embedding,
                            "num_candidates": 100,
                            "boost": 0.85
                        }
                    }
                ],
                "minimum_should_match": 1
            }
        },
        "rescore": {
            "window_size": top_k * 2,
            "query": {
                "rescore_query": {
                    "script_score": {
                        "query": {"match_all": {}},
                        "script": {
                            "source": """
                                double text_score = _score;
                                double semantic_score = 0.0;
                                if (doc['embedding'].size() == params.query_vector.size()) {
                                    semantic_score = cosineSimilarity(params.query_vector, 'embedding') + 1.0;
                                }
                                return text_score * 0.3 + semantic_score * 0.7;
                            """,
                            "params": {"query_vector": query_embedding}
                        }
                    }
                },
                "query_weight": 0.4,
                "rescore_query_weight": 0.6
            }
        },
        "_source": ["id", "productname","productname.no_accent"]
    }

    return es_client.search(index=index_name, body=search_body)


In [136]:
query_text = "ăn uống healthy"
response = hybrid_search_with_llm(query_text, es)

if response["hits"]["hits"]:
    print(f"Tìm thấy {len(response['hits']['hits'])} kết quả:\n")
    for i, hit in enumerate(response["hits"]["hits"], 1):
        print(f"#{i} | Score: {hit['_score']:.4f}")
        print(f"   Tên sản phẩm: {hit['_source'].get('productname', 'N/A')}")
else:
    print("Không tìm thấy kết quả phù hợp")


>>> Query sau khi LLM rewrite: "món ăn uống healthy"
Tìm thấy 20 kết quả:

#1 | Score: 0.7408
   Tên sản phẩm: Nước uống healthy 82x Kakemono Collagen trẻ hóa làn da vị ổi hồng không đường, không béo lon 245ml
#2 | Score: 0.7164
   Tên sản phẩm: Thức Uống Nuvi Jelly Thạch Đào túi 110ml
#3 | Score: 0.7003
   Tên sản phẩm: Bánh Sandwich Gạo Lứt Bảo Ngọc Healthy gói 215g
#4 | Score: 0.6944
   Tên sản phẩm: [SÁCH] Kỵ Và Hợp Trong Ăn Uống
#5 | Score: 0.6910
   Tên sản phẩm: [CỰC RẺ] Sữa chua uống nho Fristi chai 80ml
#6 | Score: 0.6832
   Tên sản phẩm: Sữa Chua Uống Vị Quả Mọng Bulgaria Meiji Chai 150ml
#7 | Score: 0.6790
   Tên sản phẩm: Combo 3 Túi Thức Uống Nuvi Jelly Thạch Đào Túi 110ml
#8 | Score: 0.6730
   Tên sản phẩm: Lốc 6 chai sữa chua uống Fristi hương táo chai 80ml
#9 | Score: 0.6683
   Tên sản phẩm: Dưa món Sông Hương hũ 450g
#10 | Score: 0.6670
   Tên sản phẩm: [ Lốc 6 chai ] Sữa uống Tiệt trùng Fristi hương nho chai 80ML
#11 | Score: 0.6646
   Tên sản phẩm: [ Lốc 6 chai ] Sữa